<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Sep7_Biobert_workingmodel_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
def readfile():
  path='/content/drive/My Drive/biobert-Sep6/datasets/NER/NCBI-disease/train.tsv'
  f = open(path)
  data = []
  data1=[]
  sentence=[]
  label=[]
  i=0
  for line in f:
    #print(i)
  #i+=1
  #print(line)
    #print("line",line)
  #print("line-l",len(line))
    if len(line)==0 or line[0]=="\n":
      data1.append((sentence,label))
      data=[]
      sentence=[]
      label=[]
      continue
    else:
      a=line.split()
  #print(a[0])
  #print(a[1])
      sentence.append(a[0])
      label.append(a[1])
  #print(data1)
  return data1

In [6]:
data=readfile()

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [9]:
import pandas as pd
import csv

In [10]:
#data['tags'].isnull().sum()
#data['tags'].unique()

tags=['O','B','I',"PAD"]
labels_map={}
for i,t in enumerate(tags):
  labels_map[t]=i
print(labels_map)


{'O': 0, 'B': 1, 'I': 2, 'PAD': 3}


In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [12]:
!pip3 install transformers

     |████████████████████████████████| 890kB 2.7MB/s 
     |████████████████████████████████| 890kB 41.9MB/s 
     |████████████████████████████████| 1.1MB 41.9MB/s 
     |████████████████████████████████| 3.0MB 35.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5f8e80d9afc685f7d044d4e73b27a6cae93a6fa97ccaa14982c64da1ee2bed29
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
#Apply the Bert
#use pytorch implementation of bert
import torch
#pip install --no-cache-dir torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#Dataset from a tensor or array or list or dict - TensorDataset
#DataLoader - represents a Python iterable over a dataset, with support for many datasets
#RandomSampler - A Sampler that returns random indices.
#SequentialSampler - A Sampler that returns indices sequentially.
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
#BertTokenizer -- constructs a tokenizer class - based on wordpiece tokenizer
#Bertconfg Config file for all parameters
#from transformers import BertTokenizer,BertConfig
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(level=logging.INFO)


In [14]:
#import clinicalbiobert
Clinicalbert_tokenizer=AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Clinicalbert_model=AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

INFO:filelock:Lock 140107611415272 acquired on /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c.lock


INFO:filelock:Lock 140107611415272 released on /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c.lock


INFO:filelock:Lock 140105316262968 acquired on /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


INFO:filelock:Lock 140105316262968 released on /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


INFO:filelock:Lock 140105315951840 acquired on /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8346d105db81ee98e35f4b35e14.lock


INFO:filelock:Lock 140105315951840 released on /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8346d105db81ee98e35f4b35e14.lock


In [15]:
sentences=[]
label=[]
for i in data:
  sentence=i[0]
  tags=i[1]
  tokenized_sentence=[]
  labels=[]
  for word,tag in zip(sentence,tags):
    #print(tag)
    #print(word)
    #print("words",word)
    #print("tags",tag)
    tokenized_word=Clinicalbert_tokenizer.tokenize(word)
    #print(tokenized_word)
    n_subwords=len(tokenized_word)
    tokenized_sentence.extend(tokenized_word)
    labels.extend([tag] * n_subwords)
  sentences.append(tokenized_sentence)
  label.append(labels)

In [16]:
print(len(sentences))

5424


In [17]:
print(len(label))

5424


In [18]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [19]:
MAX_LEN = 128
Batch_Size = 32

In [20]:
input_ids = pad_sequences([Clinicalbert_tokenizer.convert_tokens_to_ids(txt) for txt in sentences],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [21]:
tags = pad_sequences([[labels_map.get(l) for l in lab] for lab in label],
                     maxlen=MAX_LEN, value=labels_map["PAD"], padding="post",
                     dtype="long", truncating="post")

In [22]:
attention_masks = [[float(i!= 0.0) for i in ii] for ii in input_ids]

In [23]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.4)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.4)

In [24]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [25]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=32)

In [26]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import AutoModelForTokenClassification,AdamW,AutoConfig

In [27]:
print(len(labels_map))

4


In [28]:
from transformers import BertForTokenClassification,AdamW,AutoConfig

model1 = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

In [29]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [31]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [32]:
#grab the optimzer
optimizer=AdamW(model1.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [33]:
from transformers import get_linear_schedule_with_warmup

In [34]:
epochs = 2

In [35]:
total_steps = len(train_dataloader) * epochs

In [37]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [38]:
from sklearn.metrics import f1_score, accuracy_score


In [39]:
import random
import numpy as np

In [40]:

# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [44]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model1.train()

  for step,batch in enumerate(train_dataloader):
    print(step)
    #print(batch)

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    #print(batch.shape)
    b_input_ids,b_input_mask,b_labels=batch
    #print(b_labels)
  #always clear the gradients before doing a backward pass
    model1.zero_grad()

   #perform a forward pass
    loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model1.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    logits=logits.detach().cpu().numpy()
    label_ids=b_labels.to('cpu').numpy()
    label_ids=b_labels


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))

  pred_tags=[tags[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tags[l_i] != "PAD"]

  valid_tags=[tags[l_i] for l in true_labels for l_i in l if tags[l_i]!="PAD"]

  #print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  #print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))




EPOCH:   0%|          | 0/2 [00:00<?, ?it/s]


Training...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
Average train loss: 0.10931161696127817
 Training is completed
Running Validation... 
Validation loss: 0.12602583017638502


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



EPOCH:  50%|█████     | 1/2 [03:55<03:55, 235.14s/it]


Training...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
Average train loss: 0.10785541245165993
 Training is completed
Running Validation... 
Validation loss: 0.12602583017638502





EPOCH: 100%|██████████| 2/2 [07:51<00:00, 235.77s/it]


In [107]:
#test_sentence = "To study the safety and efficacy of  methylphenidate in children with the dual diagnosis of  epilepsy and attention deficit hyperactivity disorder  and double-blind, crossover design was used to compare the effects "
test_sentence="It was the unanimous view of the Emergency Committee that Ebola virus disease outbreak in Democratic Republic of the Congo still constitutes a public health emergency of international concern (PHEIC)"

In [127]:
def evaluate(test_sentence):
  tokenized_sentence=Clinicalbert_tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  #run the sentence through the model
  with torch.no_grad():
    output=model1(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=Clinicalbert_tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-Disease','I-Disease',"PAD"]
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label))      


In [129]:
evaluate('To study the safety and efficacy of  methylphenidate in children with the dual diagnosis of  epilepsy and attention deficit hyperactivity disorder  and double-blind, crossover design was used to compare the effects')

[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	O
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	B-Disease
and	O
attention	B-Disease
deficit	I-Disease
hyperactivity	I-Disease
disorder	I-Disease
and	O
double	B-Disease
-	I-Disease
blind	I-Disease
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	O


In [75]:
model1.save_pretrained('/content/drive/My Drive/NLP/Spe7Biobert')

In [76]:
Clinicalbert_tokenizer.save_pretrained('/content/drive/My Drive/NLP/Spe7Biobert')

('/content/drive/My Drive/NLP/Spe7Biobert/vocab.txt',
 '/content/drive/My Drive/NLP/Spe7Biobert/special_tokens_map.json',
 '/content/drive/My Drive/NLP/Spe7Biobert/added_tokens.json')

In [120]:
label_map = {i : label for i, label in enumerate(tags,1)}
print(label_map)

{1: 'O', 2: 'B-Disease', 3: 'I-Disease', 4: 'PAD'}


In [121]:
print(len(label_map))

4


In [123]:
bert_model='emilyalsentzer/Bio_ClinicalBERT'

In [125]:
import json

In [126]:
label_map = {i : label for i, label in enumerate(tags,1)}
model_config = {"bert_model":bert_model,"max_seq_length":128,"num_labels":len(label_map),"label_map":label_map}
json.dump(model_config,open(os.path.join('/content/drive/My Drive/NLP/Spe7Biobert',"model_config.json"),"w"))